In [1]:
# using ZipFile,
using CSV
using DataFrames
using StatsBase
using Random, Distributions
using Flux

In [2]:
# Import ratings file
cd(raw"/home/emmanuel/Development/Learning/capstone-movielens/")
# z = ZipFile.Reader("datasets/ratings.zip")
# ratings = CSV.read(z.files[1];
#                 delim="::",
#                 header=["userId", "movieId", "rating", "timestamp"],
#                 types = [Int, Int, Float64, Int])

edx = CSV.read("datasets/edx.csv";
               delim=",",
               copycols=true,
               datarow=2,
               header=["rowN", "userId", "movieId", "rating", "timestamp", "title", "genres"],
               types = [Int, Int, Int, Float64, Int, String, String])

validation = CSV.read("datasets/validation.csv";
                      delim=",",
                      copycols=true,
                      datarow=2,
                      header=["rowN", "userId", "movieId", "rating", "timestamp", "title", "genres"],
                      types = [Int, Int, Int, Float64, Int, String, String])

,rowN,userId,movieId,rating,timestamp,title
,Int64,Int64,Int64,Float64,Int64,String
1,1,1,231,5.0,838983392,Dumb & Dumber (1994)
2,2,1,480,5.0,838983653,Jurassic Park (1993)
3,3,1,586,5.0,838984068,Home Alone (1990)
4,4,2,151,3.0,868246450,Rob Roy (1995)
5,5,2,858,2.0,868245645,"Godfather, The (1972)"
6,6,2,1544,3.0,868245920,"Lost World: Jurassic Park, The (Jurassic Park 2) (1997)"
7,7,3,590,3.5,1136075494,Dances with Wolves (1990)
8,8,3,4995,4.5,1133571200,"Beautiful Mind, A (2001)"
9,9,4,34,5.0,844416936,Babe (1995)


In [4]:
r_m, r_sd = mean_and_std(edx.rating)

edx.rating_z        = (edx.rating .- r_m) ./ r_sd
validation.rating_z = (validation.rating .- r_m) ./ r_sd

999999-element Array{Float64,1}:
  1.4028961120912835 
  1.4028961120912835 
  1.4028961120912835 
 -0.483306635705546  
 -1.4264080096039606 
 -0.483306635705546  
 -0.01175594875633861
  0.9313454251420761 
  1.4028961120912835 
 -0.483306635705546  
 -0.483306635705546  
 -0.483306635705546  
 -0.483306635705546  
  ⋮                  
  0.45979473819286876
 -0.483306635705546  
 -0.483306635705546  
 -0.483306635705546  
  0.45979473819286876
  0.45979473819286876
 -0.483306635705546  
  1.4028961120912835 
 -0.483306635705546  
 -0.483306635705546  
  0.45979473819286876
  0.45979473819286876

In [5]:
movieIndex = DataFrame(movieId = unique(edx, :movieId).movieId)
nMovie = size(movieIndex)[1]
movieIndex.movieN = 1:nMovie

userIndex = DataFrame(userId = unique(edx, :userId).userId)
nUser = size(userIndex)[1]
userIndex.userN = 1:nUser

movieMean = by(edx, :movieId, movie_mean = :rating_z => mean)
userMean = by(edx, :userId, user_mean = :rating_z => mean)

,userId,user_mean
,Int64,Float64
1,1,1.4029
2,2,-0.205924
3,3,0.398949
4,4,0.513686
5,5,0.383327
6,6,0.411431
7,7,0.332083
8,8,-0.118779
9,9,0.504704


In [6]:
training = join(edx, movieIndex, on = :movieId )
training = join(training, userIndex, on = :userId )
training = training[:, [:userN, :movieN, :rating_z]]
nSamples = size(training)[1]

test = join(validation, movieIndex, on = :movieId )
test = join(test, userIndex, on = :userId )
test = test[:, [:userN, :movieN, :rating, :rating_z]]
nTest = size(test)[1]

999999

In [18]:
nLF = 3

P = zeros(Float64, nUser, nLF)
Q = zeros(Float64, nMovie, nLF)

P[:,1] .= 1
P[:,2] = sort(join(userIndex, userMean, on = :userId, kind = :left), :userN)[:, :user_mean]

Q[:,1] = sort(join(movieIndex, movieMean, on = :movieId, kind = :left), :movieN)[:, :movie_mean]
Q[:,2] .= 1

Random.seed!(42)
P[:, 3] = rand(Normal(), nUser) / 1000
Q[:, 3] = rand(Normal(), nMovie) / 1000


# Make P and Q Flux parameters
P = param(P)
Q = param(Q)


Tracked 10677×3 Array{Float64,2}:
 -0.616675   1.0  -0.0008925  
 -0.361332   1.0  -0.000540742
 -0.0890802  1.0  -0.00165577 
 -0.153526   1.0  -0.000116193
 -0.16505    1.0  -0.00052468 
 -0.966375   1.0  -0.000815219
  0.471887   1.0  -0.00100502 
 -0.0541851  1.0   0.000336547
  0.225951   1.0  -0.000794165
 -0.52383    1.0  -0.000984026
  0.0135865  1.0   5.96994e-5 
 -0.71666    1.0   0.000200833
 -0.575833   1.0   0.000602939
  ⋮                           
  0.459795   1.0  -0.000875203
 -2.84106    1.0   0.00135934 
  0.459795   1.0   0.000346972
 -1.89796    1.0   0.000240046
 -0.483307   1.0  -0.000962833
  0.459795   1.0   1.02952e-5 
 -2.84106    1.0  -0.00129574 
 -0.0117559  1.0  -0.000742125
 -0.483307   1.0   0.000798655
 -1.42641    1.0  -0.00132777 
 -1.42641    1.0  -0.00228968 
 -0.954857   1.0  -0.00110868 

In [28]:
# Cost function on sample

function J(p, q)
    error = sum( p[test[:,:userN], :] .* q[test[:,:movieN], :]; dims = 2) .* r_sd .+ r_m
    error = sqrt( sum((test.rating - error).^2) / nTest )
    
    return error
end

J (generic function with 3 methods)

In [32]:
P[13, :]

Tracked 3-element Array{Float64,1}:
  1.0                  
 -0.1988792372282461   
  0.0005181487878771377

In [31]:
# Cost function on full set

function prediction(user_n, movie_n) = sum( P[user_n, :] .* Q[movie_n, :], dims = 2) .* r_sd + r_m 
    

function J(p, q)
    error = sum( p[training[:,:userN], :] .* q[training[:,:movieN], :]; dims = 2) .* r_sd .+ r_m
    error = sqrt( sum((training.rating - error).^2) / nTest )
    
    return error
end
 

J (generic function with 3 methods)

In [30]:
# Gradient

dJ = Tracker.gradient(() -> J(p, q), Flux.params(P, Q), nest = true)


UndefVarError: UndefVarError: p not defined

In [ ]:
function stochastic_grad_descent(P::Array{Float64, 2}, Q::Array{Float64, 2}, grad;
  times = 1, batch_size = 10000, λ = 0.1, α = 0.01, verbose = true)

  for i = 1:times
        
    spl_P = P[spl_training[:,:userN], :]
    spl_Q = Q[spl_training[:,:movieN], :]
    
       
        
    P[spl_training[:,:userN], :]  = spl_P - α * ΔP
    Q[spl_training[:,:movieN], :] = spl_Q - α * ΔQ
  end # for loop

  return P, Q
end

In [ ]:
float_t = sum( P[test[:,:userN], :] .* Q[test[:,:movieN], :]; dims = 2) .* r_sd .+ r_m
float_t = sqrt( sum((test.rating - float_t).^2) / nTest )
println("Initial floating point RMSE test = ",float_t)

round_t = round.(P[test[:,:userN], :] .* Q[test[:,:movieN], :])
round_t = sum(round_t ; dims = 2) .* r_sd .+ r_m
round_t = sqrt( sum((test.rating - round_t).^2) / nTest )
println("Initial rounded RMSE test = ", round_t)

In [ ]:
starting_α = 0.01
validation_results =[0 float_t round_t]

for n = 1:500
  batch_size = 10000
  α = starting_α
  nFeatures = size(P)[2]
  λ = 0.1 * (nUser + nMovie) * nFeatures / 2_000_000

  RMSE = sqrt(sum((training.rating_z -
                  sum( P[training[:,:userN], :] .* Q[training[:,:movieN], :]; dims = 2)).^2) / nSamples)

  for i = 1:250
    RMSE_tmp =   RMSE

    (newP, newQ) = stochastic_grad_descent(P, Q; times = 100, batch_size = 1000 * nFeatures, λ = λ, α = α)
    global P = newP
    global Q = newQ

    RMSE = sqrt(sum((training.rating_z -
                     sum( P[training[:,:userN], :] .* Q[training[:,:movieN], :]; dims = 2)).^2) / nSamples)

    println("Step: ", i, "   RMSE of z-score training = ", RMSE)

    if (RMSE > RMSE_tmp)
      α /= 2
      println("α decreased to ", α)
    end

    if (starting_α / α > 10000) | (abs((RMSE - RMSE_tmp) / RMSE_tmp) < 1e-6)
      break
    end

  end


  test_RMSE = P[test[:,:userN], :] .* Q[test[:,:movieN], :]

  # floating point ratings float_RMSE = sum(test_RMSE ; dims = 2).* r_sd .+ r_m float_RMSE = sqrt( sum((test.rating - float_RMSE).^2) / nTest ) println("Step: ", n, "    float RMSE test = ",float_RMSE, " with number features = ", nFeatures) # Round to only obtain legal ratings round_RMSE = sum( round.(test_RMSE) ; dims = 2).* r_sd .+ r_m round_RMSE = sqrt( sum((test.rating - round_RMSE).^2) / nTest ) println("Step: ", n, "    round RMSE test = ",round_RMSE, " with number features = ", nFeatures) global validation_results = [validation_results; nFeatures float_RMSE round_RMSE] # Add 1 features global P = [P rand(Normal(), nUser)/1000] global Q = [Q rand(Normal(), nMovie)/1000]
  nFeatures += 1

end

In [3]:
function stochastic_grad_descent_simple(P::Array{Float64, 2}, Q::Array{Float64, 2};
  times = 1, batch_size = 10000, λ = 0.1, α = 0.01, verbose = true)

  for i = 1:times
    spl_training = view(training, sample(axes(training, 1), batch_size; replace = false, ordered = true), :)

    spl_P = P[spl_training[:,:userN], :]
    spl_Q = Q[spl_training[:,:movieN], :]

    err = spl_training.rating_z - sum(spl_P .* spl_Q; dims = 2)
    ΔP = - 2 * err .* spl_Q + λ * spl_P
    ΔQ = - 2 * err .* spl_P + λ * spl_Q

    P[spl_training[:,:userN], :]  = spl_P - α * ΔP
    Q[spl_training[:,:movieN], :] = spl_Q - α * ΔQ
  end # for loop

  return P, Q
end

stochastic_grad_descent_simple (generic function with 1 method)